# Lesson 3 - Building a RAG Agent with CrewAI

In this lesson, you will build a RAG agent with `CrewAI`. You will do that by integrating `RagTool` from `crewai_tools` with a `CrewAI` agent. `RagTool` provides a way to create and query knowledge bases from various data sources, and allows the agent to access specialized context. In this lesson, you will provide the RAG tool a pdf file containing details about insurance coverage provided by a private health insurer. By the end of the lesson, you will build an insurer agent specialized in answering queries related to health benefits. In the next lessons, you will wrap this agent in an ACP server and make it interact with other ACP agents.

## 3.1. Import Libraries

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from crewai import LLM, Agent, Crew, Task
from crewai_tools import RagTool

/home/zeyang/git-repos/ai-courses/deeplearning_ai/agent_communication_protocol/.venv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## 3.2. Define the Agent's Large Language Model

You'll now define the large language model that you will use for your CrewAI agent. `max_tokens`: maximum number of tokens the model can generate in a single response.

**Note**: If you will define this model locally, it requires that you define the API key in a **.env** file as follows:
```
# Required
OPENAI_API_KEY=sk-...

# Optional
OPENAI_API_BASE=<custom-base-url>
OPENAI_ORGANIZATION=<your-org-id>
```

In [3]:
llm = LLM(
    base_url="https://openrouter.ai/api/v1",
    model="openrouter/google/gemma-3-12b-it:free",
    timeout=60,
    temperature=0.7,
    max_tokens=1024,
    stream=False,
)

## 3.3. Define the RAG Tool

For the RAG tool, you can define the model provider and the embedding model in a configuration Python dictionary. You can also define the details of your vector database. If you don't specify the vector database, the RagTool will use Chroma (ChromaDB) as the default vector database in local/in-memory mode.

In [4]:
config = {
    "llm": {
        "provider": "ollama",
        "config": {
            "model": "gemma3:1b",
            "temperature": 0.5,
            "base_url": "http://localhost:11434",
        },
    },
    "embedding_model": {
        "provider": "ollama",
        "config": {
            "model": "all-minilm:latest",
            "base_url": "http://localhost:11434",
        },
    },
}

You can then pass the `config` to the `RagTool`, and then specify the data source for which the knowledge base will be constructed. When embedding your data, the `RagTool` chunks your document into chunks and create an embedding vector for each chunk. You can specify the chunk size (`chunk_size`: number of characters) and how many characters overlap between consecutive chunks (`chunk_overlap`). You can also use the default behavior.

In [5]:
rag_tool = RagTool(config=config, chunk_size=1200, chunk_overlap=200)

/home/zeyang/git-repos/ai-courses/deeplearning_ai/agent_communication_protocol/.venv/lib/python3.12/site-packages/alembic/config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/agent_communication_protocol/.venv/lib/python3.12/site-packages/embedchain/embedder/ollama.py:27: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=self.config.model, base_url=config.base_url)
/home/zeyang/git-repos/ai-courses/deeplearning_

In [6]:
rag_tool.add("./data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


## 3.4. Define the Insurance Agent

Now that you have the `rag_tool` defined, you define the CrewAI agent that can assist with insurance coverage queries.

In [7]:
insurance_agent = Agent(
    role="Senior Insurance Coverage Assistant",
    goal="Determine whether something is covered or not",
    backstory="You are an expert insurance agent designed to assist with coverage queries",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool],
    max_retry_limit=2,
)

## 3.5. Define the Agent Task

Let's now test the insurance agent. For that, you need to define the agent task and pass to it the query and the agent.

In [8]:
task1 = Task(
    description="What is the waiting period for rehabilitation?",
    expected_output="A comprehensive response as to the users question",
    agent=insurance_agent,
)

## 3.6. Run the Insurance Agent

To run the agent, you need to pass the agent and the task to a Crew object that you can run using the `kickoff` method.

In [9]:
crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)

task_output = crew.kickoff()

print(task_output)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2667d426-ba55-46f4-8abd-bb597386fbfd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Task: What is the waiting period for rehabilitation?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/home/zeyang/git-repos/ai-courses/deeplearning_ai/agent_communication_protocol/.venv/lib/python3.12/site-packages/c
hromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is 
deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be 
removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Using Tool: Knowledge base                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The waiting period for rehabilitation is 2 months. However, please note that if the rehabilitation is related  │
│  to a pre-existing condition, the waiting period is 12 months.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1a51162b-512a-4049-8cf3-70ebefd4b615                                                                     │
│  Agent: Senior Insurance Coverage Assistant                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2667d426-ba55-46f4-8abd-bb597386fbfd                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The waiting period for rehabilitation is 2 months. However, please note that if the              │
│  rehabilitation is related to a pre-existing condition, the waiting period is 12 months.                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The waiting period for rehabilitation is 2 months. However, please note that if the rehabilitation is related to a pre-existing condition, the waiting period is 12 months.


## 3.7. Resources

- [CrewAI Agents](https://docs.crewai.com/concepts/agents)
- [CrewAI Tasks](https://docs.crewai.com/concepts/tasks)
- [CrewAI RagTool](https://docs.crewai.com/tools/ai-ml/ragtool)
- [Short course on Multi Agents with CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)